In [ ]:
pip install selenium

# Bibliotecas

In [ ]:
import pandas as pd

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# Criar Uma Instância do Chrome

In [ ]:
service = Service()

options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--headless")
options.add_argument("--disable-gpu")

driver = webdriver.Chrome(service=service, options=options)

# Coletar Dados

Navegar até o site do DATASUS

In [ ]:
url = 'http://tabnet.datasus.gov.br/cgi/deftohtm.exe?sih/cnv/spabr.def'

driver.get(url)

Coletar Linhas da Consulta Gerada na Nova Aba e Concatená-las

In [ ]:
lin = ""
arqs = []
colunas_df = ['Município', 
           '01 Ações de promoção e prevenção em saúde',
           '02 Procedimentos com finalidade diagnóstica',
           '03 Procedimentos clínicos',
           '04 Procedimentos cirúrgicos',
           '05 Transplantes de orgãos, tecidos e células',
           '06 Medicamentos',
           '07 Órteses, próteses e materiais especiais',
           '08 Ações complementares da atenção à saúde',
           'QTD TOTAL']


# Em "Coluna" Selecionar a opção "Grupo Procedimento"
driver.find_element(By.XPATH, '//*[@id="C"]/option[7]').click()

# Deselecionar o Periodo
driver.find_element(By.XPATH, f'//*[@id="A"]/option[1]').click()

for periodo in range(192):   
    
    # Selecionar Periodo
    driver.find_element(By.XPATH, f'//*[@id="A"]/option[{periodo+1}]').click()
    data = driver.find_element(By.XPATH, f'//*[@id="A"]/option[{periodo+1}]').text + ','
    print('Coletando dados de', data)
    
    # Selecionar 'Colunas Separadas por ";"'
    driver.find_element(By.XPATH, '/html/body/div/div/center/div/form/div[4]/div[2]/div[1]/div[2]/input[3]').click()
    
    # Clicar em Mostrar
    driver.find_element(By.XPATH, '/html/body/div/div/center/div/form/div[4]/div[2]/div[2]/input[1]').click()
    
    # Ir até nova aba
    janelas_abertas = driver.window_handles
    driver.switch_to.window(janelas_abertas[1])
    
    print('Gerando e Salvando Dataframe')
    # Gerar e Salvar Dataframe
    linhas = driver.find_element(By.XPATH, '/html/body/div/div/pre').text.split('\n')
    colunas = linhas[0].split(';')
    dados = [linha.split(';') for linha in linhas[1:]]
    df = pd.DataFrame(dados, columns=colunas)
    df['Data'] = data
    arqs.append(df)
    
    # Fechar Aba atual e navegar até primeira aba
    driver.close()
    janelas_abertas = driver.window_handles
    driver.switch_to.window(janelas_abertas[0])    
    
    # Deselecionar Periodo
    driver.find_element(By.XPATH, f'//*[@id="A"]/option[{periodo+1}]').click()
    
#Concatenar arquivos
df_resultado = pd.concat(arqs, ignore_index=True)

# Exportar Arquivo
df_resultado.to_csv('procedimentos_datasus_16_anos.csv', index=False)

# Tratar Dados

In [ ]:
df = pd.read_csv('procedimentos_datasus_16_anos.csv', low_memory=False)

In [ ]:
df.info()

In [ ]:
def remover_aspas(valor):
    if isinstance(valor, str):
        return valor.replace('"', '')
    else:
        return valor

# Aplica a função a cada elemento do DataFrame

for coluna in df.columns:
    if df[coluna].dtype != 'float64':
        df[coluna] = df[coluna].apply(remover_aspas) 

In [ ]:
df.head()

In [ ]:
df['Cod_Município'] = df['"Município"'].str.slice(0, 6)

In [ ]:
df.head()

In [ ]:
df['"Município"'] = df['"Município"'].str.slice(6)

In [ ]:
df.head()

In [ ]:
df[['Mês', 'Ano']] = df["Data"].str.split('/', expand=True)

In [ ]:
df.head()

In [ ]:
df['Ano'] = df['Ano'].apply(lambda x: x.replace(',', ''))

In [ ]:
df.head()

In [ ]:
df = df.drop('Data', axis=1)

In [ ]:
df.head()

In [ ]:
df.columns = [coluna.replace('"', '') for coluna in df.columns]

In [ ]:
df.columns

In [ ]:
df = df[['Ano','Mês','Cod_Município','Município','01 Ações de promoção e prevenção em saúde',
       '02 Procedimentos com finalidade diagnóstica',
       '03 Procedimentos clínicos', '04 Procedimentos cirúrgicos',
       '05 Transplantes de orgãos, tecidos e células', '06 Medicamentos',
       '07 Órteses, próteses e materiais especiais',
       '08 Ações complementares da atenção à saúde', 'Total']]

In [ ]:
df.head()

In [ ]:
df.to_csv('procedimentos_datasus_16_anos_tratados.csv')